In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [31]:
df = pd.read_csv("Human Microbiome.csv")
df.head()

,HMP ID,GOLD ID,Organism Name,Domain,NCBI Superkingdom,HMP Isolation Body Site,Project Status,Current Finishing Level,NCBI Submission Status,NCBI Project ID,Genbank ID,Gene Count,IMG/HMP ID,HOMD ID,Sequencing Center,Funding Source,Strain Repository ID
0,1,Gi03551,Abiotrophia defectiva ATCC 49176,BACTERIAL,Bacteria,oral,Complete,Level 3: Improved-High-Quality Draft,6. annotation (and sequence) public on NCBI site,33011,ACIN00000000,1950,643886181,HOMD: tax_389,Washington University Genome Sequencing Center,NIH-HMP Jumpstart Supplement,"ATCC 49176, CIP 103242"
1,4,Gi03555,Achromobacter piechaudii ATCC 43553,BACTERIAL,Bacteria,airways,Complete,Level 2: High-Quality Draft,6. annotation (and sequence) public on NCBI site,46343,ADMS00000000,5755,647000200,NaN,Baylor College of Medicine,NIH-HMP Jumpstart Supplement,"ATCC 43553, CIP 55774, LMG 6100"
2,5,Gi03554,Achromobacter xylosoxidans C54,BACTERIAL,Bacteria,airways,Complete,Level 5: Non-contiguous Finished,6. annotation (and sequence) public on NCBI site,38739,ACRC00000000,6010,0,HOMD: tax_343,Broad Institute,NIH-HMP Jumpstart Supplement,BEI HM-235
3,10,Gi03422,Acinetobacter baumannii ATCC 19606,BACTERIAL,Bacteria,urogenital_tract,Complete,Level 2: High-Quality Draft,6. annotation (and sequence) public on NCBI site,38509,ACQB00000000,3832,647533101,HOMD: tax_554,Broad Institute,NIH-HMP Jumpstart Supplement,"ATCC 19606, DSM 6974"
4,12,Gi03421,Acinetobacter calcoaceticus RUH2202,BACTERIAL,Bacteria,skin,Complete,Level 2: High-Quality Draft,6. annotation (and sequence) public on NCBI site,38337,ACPK00000000,3632,646206267,NaN,Broad Institute,NIH-HMP Jumpstart Supplement,LMG 10517


In [33]:
df_selected = df[[
    "Organism Name",
    "Domain",
    "NCBI Superkingdom",
    "HMP Isolation Body Site"
]].copy()

df_selected.head()

,Organism Name,Domain,NCBI Superkingdom,HMP Isolation Body Site
0,Abiotrophia defectiva ATCC 49176,BACTERIAL,Bacteria,oral
1,Achromobacter piechaudii ATCC 43553,BACTERIAL,Bacteria,airways
2,Achromobacter xylosoxidans C54,BACTERIAL,Bacteria,airways
3,Acinetobacter baumannii ATCC 19606,BACTERIAL,Bacteria,urogenital_tract
4,Acinetobacter calcoaceticus RUH2202,BACTERIAL,Bacteria,skin


In [35]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cluster_range = range(2, 11)

In [36]:
def make_ohe():
    try:
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown='ignore', sparse=False)

categorical_cols = df_selected.columns.tolist()

preprocess = ColumnTransformer([
    ("cat", make_ohe(), categorical_cols)
])

In [37]:
grid_results = {}

for k in cluster_range:
    pipe = Pipeline([
        ("preprocess", preprocess),
        ("pca", PCA(n_components=2)),
        ("kmeans", KMeans(n_clusters=k, random_state=42))
    ])

    scores = []

    for train_idx, test_idx in kf.split(df_selected):
        train = df_selected.iloc[train_idx]
        test  = df_selected.iloc[test_idx]
        pipe.fit(train)
        clusters = pipe.predict(test)

        transformed = pipe.named_steps['pca'].transform(
            pipe.named_steps['preprocess'].transform(test)
        )

        try:
            if len(np.unique(clusters)) < 2:
                score = np.nan
            else:
                _, c = np.unique(clusters, return_counts=True)
                if (c < 2).any():
                    score = np.nan
                else:
                    score = silhouette_score(transformed, clusters)
        except:
            score = np.nan
        
        scores.append(score)

    valid = [s for s in scores if not np.isnan(s)]
    avg = np.nan if len(valid)==0 else sum(valid)/len(valid)
    
    grid_results[k] = avg
    print(f"k={k}  avg silhouette = {avg}")

k=2  avg silhouette = 0.5240626849641341
k=3  avg silhouette = 0.70117253162247
k=4  avg silhouette = 0.8522538134180945
k=5  avg silhouette = 0.8902175287171362
k=6  avg silhouette = 0.931134710700638
k=7  avg silhouette = 0.9337527354855183
k=8  avg silhouette = 0.9327731420008001
k=9  avg silhouette = 0.8963856110772515
k=10  avg silhouette = 0.8878694794783056


In [38]:
best_k = max(grid_results, key=grid_results.get)
best_k, grid_results[best_k]

(7, np.float64(0.9337527354855183))

In [39]:
final_pipe = Pipeline([
    ("preprocess", preprocess),
    ("pca", PCA(n_components=2)),
    ("kmeans", KMeans(n_clusters=best_k, random_state=42))
])

final_pipe.fit(df_selected)
clusters = final_pipe.predict(df_selected)

df_selected["Cluster"] = clusters
df_selected.head()

,Organism Name,Domain,NCBI Superkingdom,HMP Isolation Body Site,Cluster
0,Abiotrophia defectiva ATCC 49176,BACTERIAL,Bacteria,oral,3
1,Achromobacter piechaudii ATCC 43553,BACTERIAL,Bacteria,airways,3
2,Achromobacter xylosoxidans C54,BACTERIAL,Bacteria,airways,3
3,Acinetobacter baumannii ATCC 19606,BACTERIAL,Bacteria,urogenital_tract,0
4,Acinetobacter calcoaceticus RUH2202,BACTERIAL,Bacteria,skin,3
